---

# Derin Öğrenme Model Mimarisi: Evrişimsel Sinir Ağları (CNN) Uygulaması

Bu ders notu, modern bir Evrişimsel Sinir Ağı (CNN) modelinin temel bileşenlerini ve çalışma prensiplerini açıklamaktadır. Sağlanan Python kodu üzerinden katmanların işlevlerini ve mimarinin genel yapısını inceleyeceğiz.

## Model Mimarisine Genel Bakış

Bu model, Keras Functional API kullanılarak oluşturulmuş bir **Xception** benzeri mimaridir. Özellikle görüntü sınıflandırma görevleri için tasarlanmıştır. Modelin temel amacı, girdi görüntülerinden anlamlı özellikler çıkararak doğru sınıflandırma tahmini yapmaktır.

---

## Model Katmanları ve Fonksiyonları

Aşağıda, model kodunda kullanılan her bir katman ve işlevinin detaylı açıklaması bulunmaktadır:

### 1. Giriş Katmanı (Input Layer)

In [ ]:
inputs = keras.Input(shape=input_shape)

* **Açıklama:** Modelin alacağı verinin şeklini tanımlar. `input_shape`, genellikle `(yükseklik, genişlik, kanal_sayısı)` formatında olur (örneğin, `(180, 180, 3)` renkli bir resim için). Bu katman, modelin beklediği girdi formatını belirler.

### 2. Yeniden Ölçeklendirme (Rescaling)

In [ ]:
x = layers.Rescaling(1.0 / 255)(inputs)

* **Açıklama:** Girdi görüntülerini piksel değerleri genellikle 0-255 arasında değişir. Bu katman, değerleri **[0, 1]** aralığına normalize eder.
* **Neden Kullanılır?** Bu **standardizasyon**, sinir ağlarının daha kararlı ve hızlı öğrenmesini sağlar, çünkü çok büyük veya çok küçük değerler gradyan patlaması/sönümlenmesi gibi sorunlara yol açabilir.

### 3. Evrişimsel Katman (Conv2D)

In [ ]:
x = layers.Conv2D(128, 3, strides=2, padding="same")(x)

* **Açıklama:** Görüntüdeki yerel desenleri ve **özellikleri (kenarlar, köşeler, dokular)** çıkarmak için kullanılır. `128` farklı **filtre (kernel)** ile girdi üzerinde tarama yapar. `3` filtre boyutu (3x3).
* **`strides=2`:** Her adımda 2 piksel atlayarak ilerler, bu da çıktı boyutunu **yarıya indirir**.
* **`padding="same"`:** Çıktı özelliğinin boyutunun, girdi özelliğinin boyutuna (stride dikkate alınarak) "aynı" kalmasını sağlamak için kenarlara dolgu (padding) ekler.

### 4. Toplu Normalizasyon (BatchNormalization)

In [ ]:
x = layers.BatchNormalization()(x)

* **Açıklama:** Her bir eğitim **mini-batch'i** içindeki aktivasyonların ortalamasını ve varyansını normalize eder. Yani her bir katmanın çıktısını belirli bir dağılıma (genellikle ortalama 0, varyans 1) sahip olacak şekilde ayarlar.
* **Neden Kullanılır?** Bu, **iç kovaryat kaymasını (internal covariate shift)** azaltarak modelin daha hızlı eğitilmesine, daha kararlı olmasına ve daha iyi genelleme yapmasına yardımcı olur. Aşırı uyumu (overfitting) bir miktar azaltmaya da yardımcı olabilir.

### 5. Aktivasyon Fonksiyonu (Activation - ReLU)

In [ ]:
x = layers.Activation("relu")(x)

* **Açıklama:** **ReLU (Rectified Linear Unit)**, $f(x) = \max(0, x)$ formülüyle çalışan doğrusal olmayan bir aktivasyon fonksiyonudur. Negatif girdileri sıfıra dönüştürür, pozitif girdileri olduğu gibi bırakır.
* **Neden Kullanılır?** Sinir ağlarına **doğrusal olmayanlık** kazandırır. Bu sayede ağ, daha karmaşık ilişkileri ve desenleri öğrenebilir. ReLU, yaygın olarak kullanılır çünkü hesaplaması basittir ve gradyan sönümlenmesi sorununu azaltmaya yardımcı olur.

### 6. Ayrılabilir Evrişim (SeparableConv2D)

In [ ]:
x = layers.SeparableConv2D(size, 3, padding="same")(x)

* **Açıklama:** Standart `Conv2D` katmanına göre daha verimli bir evrişim türüdür. **Derinlemesine (Depthwise)** ve **Noktasal (Pointwise)** evrişimlerin birleşimidir.
    * **Derinlemesine Evrişim:** Her bir giriş kanalını bağımsız olarak işler.
    * **Noktasal Evrişim (1x1 Conv):** Derinlemesine evrişimin çıktısını birleştirerek yeni özellik haritaları oluşturur.
* **Neden Kullanılır?** Daha az hesaplama maliyeti ve parametre sayısı gerektirir, bu da daha derin modellerin daha hızlı eğitilmesini sağlar ve aşırı uyumu azaltmaya yardımcı olabilir.

### 7. Maksimum Havuzlama (MaxPooling2D)

In [ ]:
x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

* **Açıklama:** Özellik haritalarının boyutunu küçültür (**downsampling**). Belirtilen boyutta (`3x3`) bir pencere üzerinde en yüksek piksel değerini seçer. `strides=2` ile çıktıyı yarıya indirir.
* **Neden Kullanılır?**
    * **Hesaplama Yükünü Azaltma:** Parametre sayısını ve hesaplama maliyetini düşürür.
    * **Konumdan Bağımsızlık:** Modelin nesnelerin görüntüdeki küçük yer değişikliklerine karşı daha az hassas olmasını sağlar, yani bir özelliğin tam konumundan ziyade varlığına odaklanır.
    * **Özellik Özetleme:** En belirgin özellikleri koruyarak gereksiz bilgiyi atar.

### 8. Kalan Bağlantılar (Residual Connections / Skip Connections)

In [ ]:
residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
x = layers.add([x, residual])

* **Açıklama:** Bu kısım, **Derin Artık Ağlar (ResNet)** mimarilerinden ilham alan önemli bir özelliktir. Bir bloğun çıktısının ( `x` ) bir önceki bloğun çıktısına ( `previous_block_activation` ) eklenmesidir. `residual` bağlantısı, boyut uyumsuzluğunu gidermek için `1x1 Conv2D` kullanır.
* **Neden Kullanılır?**
    * **Gradyan Akışını Kolaylaştırma:** Çok derin ağlarda gradyanların kaybolması (vanishing gradients) sorununu hafifleterek eğitimin daha kararlı olmasını sağlar.
    * **Performansı Artırma:** Modelin daha derin katmanlarda bile etkili bir şekilde öğrenmeye devam etmesini sağlar.
    * **Kimlik Eşleme (Identity Mapping):** Blokların en azından bir kimlik eşleme öğrenmesini sağlar, yani ağırlıkları sıfır olsa bile bilgi akışı korunur.

### 9. Global Ortalama Havuzlama (GlobalAveragePooling2D)

In [ ]:
x = layers.GlobalAveragePooling2D()(x)

* **Açıklama:** Her bir özellik haritasının ortalamasını alır ve sonuç olarak her bir özellik haritası için tek bir skaler değer üretir. Uzamsal boyutları (genişlik ve yükseklik) tek bir değere sıkıştırır.
* **Neden Kullanılır?**
    * **Parametre Azaltma:** Tamamen bağlı katmanlardaki parametre sayısını önemli ölçüde azaltır.
    * **Konumdan Bağımsızlık:** Modelin nesnenin uzamsal konumuna daha az bağlı olmasını sağlar.
    * **Daha İyi Genelleme:** Genellikle aşırı uyumu azaltmaya yardımcı olur.

### 10. Dropout Katmanı

In [ ]:
x = layers.Dropout(0.25)(x)

* **Açıklama:** Eğitim sırasında, rastgele seçilen nöronların belirli bir yüzdesini (burada %25'ini) geçici olarak devre dışı bırakır. Bu, bu nöronların çıktıları sıfırlanır.
* **Neden Kullanılır?**
    * **Aşırı Uyum Engelleme:** Nöronların belirli özelliklere veya diğer nöronlara aşırı bağımlı olmasını önler. Her eğitim adımında farklı bir ağ yapılandırılmış gibi olur.
    * **Model Sağlamlığı:** Modelin daha sağlam ve genellenebilir olmasını sağlar. **Yalnızca eğitim sırasında etkindir**, çıkarım (inference) aşamasında devre dışıdır.

### 11. Yoğun Katman (Dense Layer)

In [ ]:
outputs = layers.Dense(units, activation=None)(x)

* **Açıklama:** Tamamen bağlantılı (Fully Connected) katmandır. Girdideki tüm nöronlar, çıktıdaki tüm nöronlara bağlıdır.
    * `units`: Sınıflandırma problemi türüne göre değişir:
        * **İkili Sınıflandırma (`num_classes == 2`):** `units = 1`. Genellikle sigmoid aktivasyon ile birleştirilir (ancak burada `activation=None` olduğu için logit döndürülür).
        * **Çoklu Sınıflandırma (`num_classes > 2`):** `units = num_classes`. Genellikle softmax aktivasyon ile birleştirilir (yine burada logit döndürülür).
* **`activation=None`:** Bu, katmanın doğrudan **logitleri** (ham, normalize edilmemiş skorları) döndüreceği anlamına gelir. Kayıp fonksiyonu (örneğin `BinaryCrossentropy(from_logits=True)` veya `CategoricalCrossentropy(from_logits=True)`) bu logitleri kullanarak doğru bir şekilde kaybı hesaplar.

---

## Modelin Akışı (Xception Benzeri Mimari)

Bu model, Google tarafından geliştirilen **Xception** mimarisinin temel prensiplerini yansıtır:

1.  **Giriş Bloğu (Entry Flow):** İlk evrişim ve batch normalizasyon katmanları ile başlar. Resimleri ön işlemden geçirir ve temel özellikleri çıkarmaya başlar.
2.  **Orta Akış Döngüsü (Middle Flow):** Modelin ana öğrenme bölümüdür. Art arda tekrarlanan ayrılabilir evrişim blokları ve aralarındaki kalan bağlantılar (residual connections) sayesinde derinlemesine özellik öğrenimi sağlanır. Her blok sonunda boyutu küçültmek için `MaxPooling2D` kullanılır.
3.  **Çıkış Bloğu (Exit Flow):** Son ayrılabilir evrişim katmanları, ardından global ortalama havuzlama ve dropout ile modelin nihai özellikleri özetlenir ve aşırı uyum önlenir. En son yoğun katman, nihai sınıflandırma tahminlerini yapar.

---

Bu yapı, Dr. Murat Altun'un da vurguladığı gibi, derin öğrenme modellerinin verimli bir şekilde eğitilmesini ve karmaşık görüntü tanıma görevlerinde yüksek başarı elde etmesini sağlar. Özellikle **derin öğrenme modellerinin nasıl tasarlandığı, katmanların ne işe yaradığı ve neden kullanıldığı** gibi temel sorulara yanıt vermektedir.